In [ ]:
!pip install tensorflow

# Importación de librerías y comprobación de GPU

In [4]:
import tensorflow as tf

import numpy as np
import os
import time

In [5]:
print("GPU Disponible:", tf.config.list_physical_devices('GPU'))

GPU Disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [6]:
!nvidia-smi

Fri Nov  7 19:32:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Carga de datos

In [7]:
import pandas as pd

# Cargar el archivo
df = pd.read_csv("/kaggle/input/southparklines/All-seasons.csv")

# Si quieres ver las columnas:
print(df.columns)

# Quedarte solo con la columna de diálogos
lines = df["Line"]

# Limpiar saltos de línea y espacios extra
lines = lines.str.replace(r'\n', ' ', regex=True).str.strip()

# Concatenar todo en un solo string
southParkDialog = str(" ".join(lines))

print(southParkDialog[:500])

Index(['Season', 'Episode', 'Character', 'Line'], dtype='object')
You guys, you guys! Chef is going away. Going away? For how long? Forever. I'm sorry boys. Chef said he's been bored, so he joining a group called the Super Adventure Club. Wow! Chef?? What kind of questions do you think adventuring around the world is gonna answer?! What's the meaning of life? Why are we here? I hope you're making the right choice. I'm gonna miss him.  I'm gonna miss Chef and I...and I don't know how to tell him! Dude, how are we gonna go on? Chef was our fuh...f-ffriend. And w


In [1]:
hola = input('ingresa valor')

ingresa valor 5


In [8]:
# Cargar el archivo
df = pd.read_csv("/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv")

# Si quieres ver las columnas:
print(df.columns)

# Quedarte solo con la columna de diálogos
lines = df["question"]

# Limpiar saltos de línea y espacios extra
lines = lines.str.replace(r'\n', ' ', regex=True).str.strip()

# Concatenar todo en un solo string
conversation = str(" ".join(lines))

print(conversation[:500])

Index(['Unnamed: 0', 'question', 'answer'], dtype='object')
hi, how are you doing? i'm fine. how about yourself? i'm pretty good. thanks for asking. no problem. so how have you been? i've been great. what about you? i've been good. i'm in school right now. what school do you go to? i go to pcc. do you like it there? it's okay. it's a really big campus. good luck with school. how's it going? i'm doing well. how about you? never better, thanks. so how have you been lately? i've actually been pretty good. you? i'm actually in school right now. which school 


In [13]:
# Read, then decode for py2 compat.
chatbot = open("/kaggle/input/chatbot-training-dataset/chatbot dataset.txt", 'rb').read(1000000).decode(encoding='utf-8')
human = open("/kaggle/input/human-conversation-training-data/human_chat.txt", 'rb').read(1000000).decode(encoding='utf-8')
wikipedia = open("/kaggle/input/wikipedia-sentences/wikisent2.txt", 'rb').read(150000).decode(encoding='utf-8')


text = chatbot+human+wikipedia[:150000]+southParkDialog[:150000]+conversation

# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 535415 characters


In [14]:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [4]:
# # Read, then decode for py2 compat.
# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# # length of text is the number of characters in it
# print(f'Length of text: {len(text)} characters')


Length of text: 1115394 characters


In [15]:
print(text[:250])

What are your interests	I am interested in all kinds of things. We can talk about anything!
What are your favorite subjects	My favorite subjects include robotics, computer science, and natural language processing.
What are your interests	I am interes


In [16]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

114 unique characters


# Procesamiento de datos


In [17]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [18]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[64, 65, 66, 67, 68, 69, 70], [87, 88, 89]]>

In [19]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [20]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)#.numpy()

text_from_ids(ids)

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'abcdefg', b'xyz'], dtype=object)>

In [21]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(535415,), dtype=int64, numpy=array([56, 71, 64, ..., 77, 67, 17])>

# Processing Data with `tf.data.Dataset`

Instead of loading the entire text into memory, we can use `tf.data.Dataset` to process the text in smaller chunks. This is especially useful for large files like the one you are using.

In [ ]:
# # Create a tf.data.Dataset from the text
# # Instead of TextLineDataset, we create a dataset of characters
# all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
# all_ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

# # You can now iterate over the dataset to process the IDs in chunks
# # For example, to see the first few elements:
# for ids in all_ids_dataset.take(5):
#   print(ids)

In [29]:
# for ids in ids_dataset.take(10):
#     print(text_from_ids(ids).numpy())

b'0.000123, which corresponds to a distance of 705 Mly, or 216 Mpc.'
b'000webhost is a free web hosting service, operated by Hostinger.'
b'0010x0010 is a Dutch-born audiovisual artist, currently living in Los Angeles.'
b'0-0-1-3 is an alcohol abuse prevention program developed in 2004 at Francis E. Warren Air Force Base based on research by the National Institute on Alcohol Abuse and Alcoholism regarding binge drinking in college students.'
b'0.01 is the debut studio album of H3llb3nt, released on February 20, 1996 by Fifth Colvmn Records.'
b'001 of 3 February 1997, which was signed between the Government of the Republic of Rwanda, and FAPADER.'
b'003230 is a South Korean food manufacturer.'
b'0.04%Gas molecules in soil are in continuous thermal motion according to the kinetic theory of gasses, there is also collision between molecules - a random walk.'
b'0.04% of the votes were invalid.'
b'005.1999.06 is the fifth studio album by the South Korean singer and actress Uhm Jung-hwa.'


In [22]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [23]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [24]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [25]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [26]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(535415,), dtype=int64, numpy=array([56, 71, 64, ..., 77, 67, 17])>

In [27]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [28]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

W
h
a
t
 
a
r
e
 
y


In [29]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'W' b'h' b'a' b't' b' ' b'a' b'r' b'e' b' ' b'y' b'o' b'u' b'r' b' '
 b'i' b'n' b't' b'e' b'r' b'e' b's' b't' b's' b'\t' b'I' b' ' b'a' b'm'
 b' ' b'i' b'n' b't' b'e' b'r' b'e' b's' b't' b'e' b'd' b' ' b'i' b'n'
 b' ' b'a' b'l' b'l' b' ' b'k' b'i' b'n' b'd' b's' b' ' b'o' b'f' b' '
 b't' b'h' b'i' b'n' b'g' b's' b'.' b' ' b'W' b'e' b' ' b'c' b'a' b'n'
 b' ' b't' b'a' b'l' b'k' b' ' b'a' b'b' b'o' b'u' b't' b' ' b'a' b'n'
 b'y' b't' b'h' b'i' b'n' b'g' b'!' b'\n' b'W' b'h' b'a' b't' b' ' b'a'
 b'r' b'e' b' '], shape=(101,), dtype=string)


In [30]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'What are your interests\tI am interested in all kinds of things. We can talk about anything!\nWhat are '
b'your favorite subjects\tMy favorite subjects include robotics, computer science, and natural language '
b'processing.\nWhat are your interests\tI am interested in a wide variety of topics, and read rather a lo'
b"t.\nWhat is your number\tI don't have any number\nWhat is your number\t23 skiddoo!\nWhat is your favorite "
b"number\tI find I'm quite fond of the number 42.\nWhat can you eat\tI consume RAM, and binary digits.\nWhy"


In [31]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [32]:
dataset = sequences.map(split_input_target)

In [33]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'What are your interests\tI am interested in all kinds of things. We can talk about anything!\nWhat are'
Target: b'hat are your interests\tI am interested in all kinds of things. We can talk about anything!\nWhat are '


# Training batches

In [34]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [35]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)

    if states is None:
        # The LSTM layer will initialize states to zero by default when initial_state is None
        lstm_output, hidden_state, cell_state = self.lstm(x, training=training)
        states = [hidden_state, cell_state] # LSTM returns a list of states
        x = lstm_output # Assign the output sequences to x
    else:
        # Use the provided states
        lstm_output, hidden_state, cell_state = self.lstm(x, initial_state=states, training=training)
        states = [hidden_state, cell_state] # LSTM returns a list of states
        x = lstm_output # Assign the output sequences to x


    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [53]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 2048

model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [54]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 115) # (batch_size, sequence_length, vocab_size)


In [55]:
model.summary()

Model: "my_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (64, 100, 512)         │        58,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ((64, 100, 2048), (64, │    20,979,712 │
│                                 │ 2048), (64, 2048))     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (64, 100, 115)         │       235,635 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,274,227 (81.15 MB)

 Trainable params: 21,274,227 (81.15 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [57]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'you a different story about the same thing. three different people will give you three different sto'

Next Char Predictions:
 b"T\xf0\x9f\x98\x9e\xe2\x80\x99\xf0\x9f\x98\x90Gm\xf0\x9f\x98\x9e\xf0\x9f\x98\x86zihQwWM#,Sg:z]7RlvYtC5.x%z\xc2\xbfQ he\xf0\x9f\x98\x9ehV;\xc2\xbf\xc3\xbc\xf0\x9f\x98\x9e\xc3\xbcqI6lB1p#\xc3\xbc%\xef\xbc\x89B$G&+\xf0\x9f\x98\x86h\n2f #rV\xc2\xbf3\xf0\x9f\x98\x90[W>i\xe2\x80\x9c\ni;\xf0\x9f\x98\x82x$&\xf0\x9f\x98\x9bfh''RN\xc3\xa9.f.HM"


In [58]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [59]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 115)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.7453337, shape=(), dtype=float32)


In [60]:
tf.exp(example_batch_mean_loss).numpy()

115.04619

In [61]:
model.compile(optimizer='adam', loss=loss)

In [63]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [64]:
EPOCHS = 40

In [65]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 30s 340ms/step - loss: 3.5342
Epoch 2/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 30s 359ms/step - loss: 2.2907
Epoch 3/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 28s 335ms/step - loss: 1.9798
Epoch 4/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 341ms/step - loss: 1.7537
Epoch 5/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 348ms/step - loss: 1.5946
Epoch 6/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 342ms/step - loss: 1.4665
Epoch 7/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 343ms/step - loss: 1.3621
Epoch 8/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 348ms/step - loss: 1.2880
Epoch 9/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 344ms/step - loss: 1.2146
Epoch 10/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 344ms/step - loss: 1.1360
Epoch 11/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 347ms/step - loss: 1.0559
Epoch 12/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 346ms/step - loss: 0.9748
Epoch 13/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 346ms/step - loss: 0.8917
Epoch 14/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 346ms/step - loss: 0.8077
Epoch 15/40
82/82 ━━━━━━━━━━━━━━━━━━━━ 29s 

In [66]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states


In [67]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [68]:
start = time.time()
states = None
next_char = tf.constant(['Hello, how are you?'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80) #[0].numpy().decode('utf-8')
print('\nRun time:', end - start)

Hello, how are you? Really good, reading and warn With here. You're funny. Butters, you see to ask me again. Cinveren, everyone we like it, Girty Warcrah. Let's support for lunch (they are not going to see a movie.
Human 2: Very though I am actually a special clouse I'm out the game! Now, leave me alroha Verouse. Oh no... All right, all right, I was getting a clue like every two minutes. I got such a raging clue that I'm vice-pretty good, I'm stafting. Everyone back him before and nobody else how are you doing this? Well what are the oddings for dinner of the head guy long and show their day going? OF watch it onlings to make sure the tanks is the mesoly school 'cm with and go to hear the ball go what happened your company way on fice with a coast. Maybe I show them in the hurd right now and tell ever the warmers though the relation on the White World of Warcraft is ignorant power right now. I think you want some team ever wind of it. It has toucle most hoirs my dea head. it's not my f

In [52]:
print(result[0], '\n\n' + '_'*80)

tf.Tensor(b"Hello, how are you?! Don't you are not ready? Ready. All rich, funday of it. Jesus Christ.  Ts yes we need to work about the weekend!\nHuman 1: Oh no\xe2\x80\xa6 I see. What would I do it in Scort?\nHuman 1: Oh too. If you could before you are weirding\nHuman 1: Hi!\nHuman 2: Hi there\nHuman 1: How is it going?\nHuman 2: Goo\nHuman 1: Hi!\nHuman 2: Hi there. I hope it's delicious little chicten about things, you've been so bad beat hands.\nHuman 1:  I hope that interesting if I was.\nDo you make meeting\tyou are been up working, and sleep in commonly which am going magazine.  It doesn't what I think they're believe in 9/11 once and meet you. I... are you went into?\nHuman 2: Anything tell you have to explain to that beautiful money? I'm different's for 9/11 and then operate the World of Radio Minest Basslovelit, Chef!! We can't look in ready here too. If we don't need some good reason, that's a good name into talking any carofocking. you like you?! I'm a Good Gack as gettin